In [51]:
import requests
from requests.structures import CaseInsensitiveDict
import numpy as np
import pandas as pd
import json
import math

In [10]:
searchSchema = {
  
  "index": "string",
  "stiType": "STI_TYPE_NONE",
  "disseminated": "NONE",
  "distribution": "PUBLIC",
  "created": {
    "gt": "string",
    "gte": "string",
    "lt": "string",
    "lte": "string",
    "format": "string"
  },
  "published": {
    "gt": "string",
    "gte": "string",
    "lt": "string",
    "lte": "string",
    "format": "string"
  },
  "modified": {
    "gt": "string",
    "gte": "string",
    "lt": "string",
    "lte": "string",
    "format": "string"
  },
  "author": [
    "string"
  ],
  "center": [
    "string"
  ],
  "organization": [
    "string"
  ],
  "subjectCategory": [
    "string"
  ],
  "reportNumber": [
    "string"
  ],
  "fundingNumber": [
    "string"
  ],
  "keyword": [
    "string"
  ],
  "page": {
    "size": 100,
    "from": 0
  },
  "sort": {
    "field": "string",
    "order": "asc"
  },
  "highlight": True,
  "q": "string",
  "title": "string",
  "abstract": "string",
  "stiTypeDetails": "string"
}

In [65]:
documents = []
with open("data/Aerothermodynamics.json") as f:
    documents = json.load(f)

In [68]:
if not documents:
  url = "https://ntrs.nasa.gov/api/citations/search?skip=25"

  headers = CaseInsensitiveDict()
  headers["accept"] = "application/json"
  headers["Content-Type"] = "application/json"

  data = {
    "q": "Aerothermodynamics",
    "page":{"size":100,"from":0}
  }

  i_resp = requests.post(url, headers=headers, data=json.dumps(data))
  documents += i_resp.json()["results"]
  for i in range(100, math.ceil(i_resp.json()["stats"]["total"]), 100):
    data["page"]["from"] = i
    resp = requests.post(url, headers=headers, data=json.dumps(data))
    documents += resp.json()["results"]



In [38]:
resp.json().keys()

dict_keys(['stats', 'results', 'aggregations'])

In [49]:
resp.json()["stats"]

{'took': 172, 'total': 2692, 'estimate': False, 'maxScore': 142.17043}

In [48]:
len(resp.json()["results"])

100

In [69]:
len(documents)

2692

In [70]:
with open("data/Aerothermodynamics.json", "w") as f:
    json.dump(documents, f, indent=4)